In [1]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

In [15]:
search = arxiv.Search(
  query = "cat:math*+OR+cat:physics*+OR+cat:cs*+AND+submittedDate:[202301010000+TO+202412312359]",
  max_results=30000
 )

In [13]:
results = client.results(search)

In [18]:
for result in results:
    print(result.title)
    break

In [7]:
for res in search.results():
    print(res)
    break

/tmp/ipykernel_6822/3237532688.py:1: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for res in search.results():


In [16]:
with open("rag_build/arxiv23_24.jsonl", "w", encoding="utf-8") as f:
    for result in results:
        record = {
            "id": result.get_short_id(),        # e.g., '2301.12345'
            "title": result.title.strip(),
            "abstract": result.summary.strip()
        }
        json.dump(record, f)
        f.write("\n")

In [24]:
import arxiv
import json
from tqdm import tqdm
from datetime import datetime, timezone

CATEGORIES = [
    "cs.AI", "cs.LG", "cs.CL", "cs.CV", "cs.NE",
    "math.PR", "math.OC", "math.ST", "math.MG", "math.AP",
    "physics.optics", "physics.comp-ph", "physics.data-an", "physics.plasm-ph"
]

CATEGORY_LIMITS = {
    # 🔷 CS - Core AI/ML Focus
    "cs.LG": 4000,   # Machine Learning
    "cs.AI": 4000,   # Artificial Intelligence
    "cs.CL": 2000,   # Computation & Language (NLP)
    "cs.CV": 2000,   # Computer Vision
    "cs.NE": 1000,   # Neural and Evolutionary Computing
    "cs.CR": 1000,   # Cryptography and Security
    "cs.DS": 1000,   # Data Structures and Algorithms
    "cs.RO": 1000,   # Robotics

    # 🔷 CS - Other relevant subfields
    "cs.SE": 500,    # Software Engineering
    "cs.SI": 500,    # Social and Information Networks
    "cs.PL": 500,    # Programming Languages
    "cs.GT": 500,    # Game Theory
    "cs.MA": 500,    # Multiagent Systems

    # 🔶 Math - Focused on optimization and probability
    "math.OC": 1500,  # Optimization and Control
    "math.PR": 1000,  # Probability
    "math.ST": 500,   # Statistics

    # 🔶 Physics - Optional light coverage
    "physics.data-an": 500,  # Data Analysis
    "physics.comp-ph": 500,  # Computational Physics
}


# Submission date range
START_DATE = datetime(2023, 1, 1, tzinfo=timezone.utc)
END_DATE = datetime(2024, 12, 31, 23, 59, tzinfo=timezone.utc)

client = arxiv.Client()

results_all = []

# Loop over categories
for cat in tqdm(CATEGORY_LIMITS.keys(), desc="Fetching categories"):
    search = arxiv.Search(
        query=f"cat:{cat}",
       # max_results=CATEGORY_LIMITS[cat],
        max_results=10,
        sort_by=arxiv.SortCriterion.SubmittedDate,
    )
    for result in client.results(search):
        if START_DATE <= result.published <= END_DATE:
            results_all.append(result)

# Save filtered results
with open("arxiv23_24.jsonl", "w", encoding="utf-8") as f:
    for result in tqdm(results_all, desc="Writing to arxiv23_24.jsonl"):
        json.dump({
            "id": result.get_short_id(),
            "title": result.title.strip(),
            "abstract": result.summary.strip()
        }, f)
        f.write("\n")

Fetching categories: 100%|██████████| 18/18 [01:08<00:00,  3.81s/it]
Writing to arxiv23_24.jsonl: 0it [00:00, ?it/s]


In [25]:
from datasets import load_dataset
from tqdm.auto import tqdm
from pathlib import Path
import re, json, datetime

# ───── user constants ─────────────────────────────────────────────
MAX_ROWS   = 40_000
OUTPUT_DIR = Path("rag_build/txt")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTFILE    = OUTPUT_DIR / "arxiv23_24.txt"

CATS_RE = re.compile(r"^(math|physics|cs)\.", re.I)
DATE_CUTOFF = datetime.date(2023, 1, 1)

# ───── helper -----------------------------------------------------
def keep_row(row) -> bool:
    """Return True if the row matches domain + date filters."""
    cats = row["categories"].split()
    if not any(CATS_RE.match(c) for c in cats):
        return False
    # 'update_date' format: '2024-05-13'
    row_date = datetime.date.fromisoformat(row["update_date"])
    return row_date >= DATE_CUTOFF

# ───── main collection loop ───────────────────────────────────────
stream = load_dataset("arxiv_dataset", split="train", streaming=True)
bar    = tqdm(total=MAX_ROWS, desc="Collecting arXiv abstracts")

with OUTFILE.open("w", encoding="utf-8") as fh:
    count = 0
    for row in stream:
        if not keep_row(row):
            continue

        # write id, title, abstract
        fh.write(f"{row['id']}\t{row['title'].strip()}\n\n{row['abstract'].strip()}\n\n")
        count += 1
        bar.update(1)

        if count >= MAX_ROWS:
            break

bar.close()
print(f"✅ Saved {count} abstracts to {OUTFILE}")

README.md:   0%|          | 0.00/7.25k [00:00<?, ?B/s]

arxiv_dataset.py:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

The repository for arxiv_dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/arxiv_dataset.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


ManualDownloadError:                 The dataset arxiv_dataset with config default requires manual data.
                Please follow the manual download instructions:
                     You need to go to https://www.kaggle.com/Cornell-University/arxiv,
and manually download the dataset. Once it is completed,
a zip folder named archive.zip will be appeared in your Downloads folder
or whichever folder your browser chooses to save files to. Extract that folder
and you would get a arxiv-metadata-oai-snapshot.json file
You can then move that file under <path/to/folder>.
The <path/to/folder> can e.g. be "~/manual_data".
arxiv_dataset can then be loaded using the following command `datasets.load_dataset("arxiv_dataset", data_dir="<path/to/folder>")`.

                Manual data can be loaded with:
                 datasets.load_dataset("arxiv_dataset", data_dir="<path/to/manual/data>")

In [26]:
from datasets import load_dataset
import pandas as pd

# 1. Load the full arXiv metadata
ds = load_dataset("mwitiderrick/arXiv", split="train")  # ~1.7M rows

# 2. Convert to pandas for ease of filtering
df = pd.DataFrame(ds)

# 3. Keep only CS/AI/ML/Algo categories
target_cats = {"cs.AI","cs.LG","cs.DS","cs.CV","cs.CL","cs.NE","cs.DS","cs.DS","cs.DM"}
df = df[df["categories"].apply(lambda tags: any(cat in tags for cat in target_cats))]

# 4. Filter by date
df["published"] = pd.to_datetime(df["published"])
mask = (df["published"] >= "2023-01-01") & (df["published"] <= "2024-12-31")
df = df[mask]

# 5. Sample ~30K if needed
df = df.sample(n=30000, random_state=42)

# 6. Write out your jsonl
df[["id","title","abstract"]] \
  .to_json("arxiv23_24.jsonl", orient="records", lines=True)


README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/7169 [00:00<?, ?it/s]

%20%20%28Ab_C8GX1S2UM0.txt:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

%20%20%28Ab_OTQIB120U9.txt:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

%20%20%22Ho_4CYCI0EQS0.txt:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

%20%20Gam_LR8JLQ9WPD.txt:   0%|          | 0.00/984 [00:00<?, ?B/s]

HTTP Error 429 thrown while requesting GET https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gau_BXNNGPQFKO.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting GET https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gen_3UFV5Y7Z59.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Geo_R048XQOCPM.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting GET https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gen_FFKVXJ9VOS.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting GET https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gen_QGHJL8IWK9.txt
Retrying in 1s [Retry 1/5].


%20%20GRE_Y9BSTE79H7.txt:   0%|          | 0.00/706 [00:00<?, ?B/s]

HTTP Error 429 thrown while requesting GET https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gen_R92REAKF94.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Giv_034MICCP67.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting GET https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gam_08SBPS8X5S.txt
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gau_R4QJUUMIML.txt
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20Gen_IO5PHJ5LG2.txt
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD h

HTTPError: 429 Client Error: Too Many Requests for url: https://huggingface.co/datasets/mwitiderrick/arXiv/resolve/f61ed293f976a0b025b4004c06d41b12ecfd799c/%20%20GRB_XV5MKQXK62.txt

In [2]:
import json, gzip, subprocess, shutil, os, re, datetime
from pathlib import Path
from tqdm.auto import tqdm

# ---------------- user constants ---------------------------------
OUT_TXT   = Path("rag_build/txt/arxiv23_24.txt")
RAW_DIR   = Path("rag_build/raw")
ZIP_FILE  = RAW_DIR / "arxiv_archive.zip"
JSON_FILE = RAW_DIR / "arxiv-metadata-oai-snapshot.json"
MAX_ROWS  = 40_000

STEM_RE   = re.compile(r"^(math|physics|cs)\.", re.I)
DATE_MIN  = datetime.date(2023, 1, 1)

RAW_DIR.mkdir(parents=True, exist_ok=True)
OUT_TXT.parent.mkdir(parents=True, exist_ok=True)

# ---------------- download ZIP if needed -------------------------
from kaggle.api.kaggle_api_extended import KaggleApi

if not ZIP_FILE.exists():
    print("Downloading via Kaggle Python API …")
    api = KaggleApi()
    api.authenticate()                     # uses ~/.kaggle/kaggle.json or env-vars
    api.dataset_download_files(
        "Cornell-University/arxiv",
        path=str(RAW_DIR),
        quiet=False
    )
    # the API leaves a zip named 'arxiv.zip' in RAW_DIR
    (RAW_DIR / "arxiv.zip").rename(ZIP_FILE)

# ---------------- extract JSON if needed -------------------------
if not JSON_FILE.exists():
    print("Extracting archive … (takes a minute)")
    shutil.unpack_archive(str(ZIP_FILE), str(RAW_DIR))
    # the JSON ends up as arxiv-metadata-oai-snapshot.json in RAW_DIR

assert JSON_FILE.exists(), "arxiv JSON not found after extraction"

# ---------------- stream & filter --------------------------------
keep, bar = 0, tqdm(total=MAX_ROWS, desc="arXiv STEM abstracts")

with JSON_FILE.open("r", encoding="utf-8") as f_in, \
     OUT_TXT.open("w", encoding="utf-8") as f_out:

    for line in f_in:
        obj = json.loads(line)

        # category & date filters
        first_cat = obj["categories"].split()[0]
        if not STEM_RE.match(first_cat):
            continue
        upd_date  = datetime.date.fromisoformat(obj["update_date"])
        if upd_date < DATE_MIN:
            continue

        text = f"{obj['id']}\t{obj['title'].strip()}\n\n{obj['abstract'].strip()}\n\n"
        f_out.write(text)

        keep += 1
        bar.update(1)
        if keep >= MAX_ROWS:
            break

bar.close()
print(f"✅  Saved {keep} abstracts → {OUT_TXT}")

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv


100%|██████████| 1.44G/1.44G [00:11<00:00, 138MB/s]



Extracting archive … (takes a minute)


arXiv STEM abstracts:   0%|          | 0/40000 [00:00<?, ?it/s]

✅  Saved 40000 abstracts → rag_build/txt/arxiv23_24.txt


In [ ]:
print('tedt')